In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from kedro.extras.datasets.pickle import PickleDataSet
from kedro.config import ConfigLoader

import os, sys
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src/'))

from src.tagseg.data.acdc_dataset import AcdcDataSet
from src.tagseg.data.dmd_dataset import DmdDataSet, DmdTimeDataSet
from src.tagseg.pipelines.data_splitting.nodes import split_data

In [ ]:
conf_paths = ["../conf/base", "../conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("catalog*", "catalog*/**")

In [ ]:
dataset = PickleDataSet(filepath='../data/05_model_input/model_input.pt').load()

In [ ]:
len(dataset)

In [ ]:
plt.imshow(dataset[0][0][0], cmap='gray')

In [ ]:
loader_dict = split_data(dataset, data_params=dict(train_val_split=.75, batch_size=16))

In [ ]:
loader_train = loader_dict.get('loader_train')

In [ ]:
image = next(iter(loader_train))

In [ ]:
images, labels = image['image'], image['label'].long()

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
from monai.metrics import DiceMetric, HausdorffDistanceMetric

dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
hd_metric = HausdorffDistanceMetric(include_background=False)

In [ ]:
from monai.transforms import AsDiscrete, EnsureType, Compose

post_label = Compose([
    EnsureType("tensor", device="cpu"),
    AsDiscrete(to_onehot=2)
])

In [ ]:
test_target.shape, test_pred.shape

In [ ]:
from monai.data import decollate_batch

list(map(lambda x: x.shape, decollate_batch(test_target)))

In [ ]:
test_target = post_label(labels[38:40])
test_pred = post_label(labels[36:38])

dice_metric(y_pred=test_pred, y=test_target)
hd_metric(y_pred=test_pred, y=test_target)

0.0

In [ ]:
dice_metric.aggregate(), hd_metric.aggregate()

In [ ]:
plt.imshow(images[38, 0], cmap='gray')
plt.imshow(labels[38, 0], cmap='Reds', alpha=0.3)